In [1]:
import open3d as o3d
import numpy as np
#replace with proper io once implementation happens
filepath = './Point Clouds/set 1/brandon slow.ply'
pcd = o3d.io.read_point_cloud(filepath)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#pcd.paint_uniform_color([0.4,0.9,0.2])
#o3d.visualization.draw_geometries([pcd])

In [41]:
#Error handling will come when this notebook is converted into a script.
if(pcd.has_points() == False):
    print("There was a problem while opening the Point Cloud file.")
else:
    print(pcd)
#print(np.asarray(pcd.points))


PointCloud with 1001813 points.


In [42]:
#Floor remover: find plane and cut everything below it (Note: nothing ensures the selected plane is horizontal)
print("Processing...")
plane_model, inliers = pcd.segment_plane(distance_threshold=0.003,
                                         ransac_n=5,
                                         num_iterations=2000)
#assemble inlier cloud (contains the segmented plane)
inlier_cloud = pcd.select_by_index(inliers)

#Get points for bounding box of both inliers and the cloud
box = pcd.get_axis_aligned_bounding_box()
inlierbox = inlier_cloud.get_axis_aligned_bounding_box()
bounds = np.asarray(box.get_box_points())
inlierbounds = np.asarray(inlierbox.get_box_points())
bounds = bounds[bounds[:,2].argsort()]
inlierbounds = inlierbounds[inlierbounds[:,2].argsort()]
#Change bounds of inlier cloud to the lower bound of the main cloud, and then crop
for i in range(4):
    bounds[(i+4)][2]= inlierbounds[(i+4)][2]
cpcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(bounds)), invert=True)
print("Floor Cropped")
temp = cpcd

Processing...
Floor Cropped


In [61]:
cpcd = temp
#Point Cloud Denoising: not sophisticated. Better methods than running the same function with a bunch of different parameters exist
print("Beginning Denoise...")
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=15,
                                                    std_ratio=1.9)
print(cpcd)
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=1.7)
print(cpcd)
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=25,
                                                    std_ratio=1.6)
print(cpcd)
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=100,
                                                    std_ratio=1.2)
print(cpcd)
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=85,
                                                    std_ratio=1.3)
print(cpcd)
print("Denoise Complete")

Beginning Denoise...
PointCloud with 166972 points.
PointCloud with 162870 points.
PointCloud with 162285 points.
PointCloud with 160393 points.
PointCloud with 159427 points.
Denoise Complete


In [62]:
cloud_low_bound = cpcd.get_min_bound()[2]
o3d.visualization.draw_geometries([cpcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [24]:
#Reconstruction: algorithm requires normals
print("Generating Model...")
cpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
d = 7
pmesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(cpcd, depth=d, linear_fit=True)
print("Generation Complete") 
print("Final Cleanup...")
#Unsophisticated Extra trimming: cuts up the feet. revision needed.
meshbounds = np.asarray(pmesh.get_axis_aligned_bounding_box().get_box_points())
meshbounds = meshbounds[meshbounds[:,2].argsort()]
for i in range(4):
    meshbounds[i][2] = cloud_low_bound
cpmesh = pmesh.crop(o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(meshbounds)))

In [25]:
#Temporary Output.
print(cpmesh)
cpmesh.paint_uniform_color([0.1,0.7,0.5])
o3d.visualization.draw_geometries([cpmesh], mesh_show_wireframe=True)

TriangleMesh with 4661 points and 9003 triangles.
